In [1]:
pip install --upgrade MetaTrader5

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone, date
from scipy.signal import find_peaks
import MetaTrader5 as mt5
from zoneinfo import ZoneInfo

In [3]:
# Initialize MetaTrader 5 connection
if not mt5.initialize():
    print("Initialization failed")
    quit()

DEFINING FUNCTIONS

In [4]:
#DROP THE UNNCESSARY COLUMNS IN THE DATAFRAME AND CONVERTING THE TIME TO HUMAN READABLE FORMAT
def df_convert_hr(rates_frame):
    rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    rates_frame.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    rates_frame["time"] = pd.to_datetime(rates_frame["time"])
    return rates_frame

#Detect the entry drills
def sell_entry_drill():
    pass

def buy_entry_drill():
    pass

DOWNLOAD THE OHLC DATA FROM THE LAST 7 DAYS

In [68]:
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M1

#GETTING THE TIMESTAMP FOR TODAY AND 3 DAYS AGO
utc_plus_2 = timezone(timedelta(hours=2))
utc_from = datetime.now(utc_plus_2) - timedelta(days=7)
utc_to = datetime.now(utc_plus_2) + timedelta(days=1)
startdate=utc_from.strftime('%Y%m%d')
enddate=utc_to.strftime('%Y%m%d')

# Get the OHLC data
rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
# Check if the data is downloaded successfully
if rates is None:
    print("No data available, error code =", mt5.last_error())
else:
    # Convert the data to a pandas DataFrame
    rates_frame = pd.DataFrame(rates)
    df_convert_hr(rates_frame)


In [69]:
rates_frame

,time,open,high,low,close
0,2024-12-30 13:32:00,1.04470,1.04471,1.04461,1.04463
1,2024-12-30 13:33:00,1.04463,1.04480,1.04462,1.04472
2,2024-12-30 13:34:00,1.04472,1.04482,1.04470,1.04471
3,2024-12-30 13:35:00,1.04470,1.04477,1.04464,1.04476
4,2024-12-30 13:36:00,1.04476,1.04480,1.04474,1.04476
...,...,...,...,...,...
5874,2025-01-06 15:27:00,1.04267,1.04335,1.04267,1.04312
5875,2025-01-06 15:28:00,1.04315,1.04343,1.04299,1.04330
5876,2025-01-06 15:29:00,1.04330,1.04365,1.04328,1.04355
5877,2025-01-06 15:30:00,1.04356,1.04356,1.04299,1.04339


DETERMINE THE MOST RECENT HIGHEST/LOWEST POINTS and THE TREND FROM THE LAST 35 MINS 

In [70]:
#Determine the max and min value of the highs and lows for the last 35 mins
Last_35 = rates_frame.tail(35)
MaxHighPrice = Last_35["high"].max()
MinLowPrice = Last_35["low"].min()

#Determine the if the last 35 mins is in a downtrend or uptrend
HighestPoint = rates_frame[rates_frame["high"] == MaxHighPrice].iloc[-1]
LowestPoint = rates_frame[rates_frame["low"] == MinLowPrice].iloc[-1]

if HighestPoint["time"] > LowestPoint["time"]:
    trend_35 = "uptrend"
elif HighestPoint["time"] < LowestPoint["time"]:
    trend_35 = "downtrend"
else:
    trend_35 = "sideways"

DETERMINE THE SESSION LOW/HIGH OF PRICE

In [74]:
date_today = date.today()

#HAVE THE DATAFRAME FOR THE CURRENT LONDON SESSION and LOOK FOR THE LONDON HIGH AND LOW:
ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 10) & (rates_frame['time'].dt.hour < 14)]
ldn_high = ldn_rates_frame["high"].max()
ldn_low = ldn_rates_frame["low"].min()

#HAVE THE DATAFRAME FOR THE CURRENT NEW YORK SESSION and LOOK FOR THE NY HIGH AND LOW: 
ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 15) & (rates_frame['time'].dt.hour < 19)]
ny_high = ny_rates_frame["high"].max()
ny_low = ny_rates_frame["low"].min()